# Importación de librerías para conexión con la base de datos en PostgresSQL

In [1]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os
import numpy as np

# Importación de librerías para entrenamiento del modelo

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Acceso a la base de datos de PostgresSQL

In [3]:
# Cargar las variables de entorno desde un archivo .env
load_dotenv()

# Obtener la URL de la base de datos desde las variables de entorno
DATABASE_URL = os.getenv("DATABASE_URL")

# Establecer una conexión a la base de datos
conn = psycopg2.connect(DATABASE_URL)

# Definir la consulta SQL para seleccionar todos los datos de la tabla "datos"
query = "SELECT * FROM datos"

# Utilizar pandas para ejecutar la consulta y cargar los resultados en un DataFrame
df = pd.read_sql_query(query, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Ahora, df contiene los datos de la tabla "datos" en un DataFrame. Comprobamos la cantidad de registros
len(df)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_33240\1929048143.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


383

# Preparación de los datos para entrenar al modelo

In [4]:
# Define una función para convertir la lista anidada en un numpy.ndarray
def parse_vector(vector_list):
    # Convierte la lista en un numpy.ndarray y luego redimensiona a (28, 28)
    return np.array(vector_list).reshape(28, 28).astype(np.float32)

# Aplica la función a la columna "vector" y crea una nueva columna "VectorNum"
df['VectorNum'] = df['vector'].apply(parse_vector)

# Convierte la columna 'VectorNum' en una lista de numpy.ndarray
train_vectors = df['VectorNum'].to_list()

# Convierte la lista en un numpy.ndarray de forma (n, 28, 28), donde n es el número de filas
train_vectors = np.array(train_vectors)

labels = df['etiqueta'].tolist()
labels = np.array(labels)

# Entrenamiento de retrained_model.h5 únicamente con los datos extraídos de la app (alojados en tabla datos, en la db de PostrgresSQL)

In [5]:
# Crear un modelo secuencial
retrained_model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

# Compilar el modelo
retrained_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
retrained_model.fit(np.array(train_vectors), labels, epochs=6)

Epoch 1/6
12/12 [==============================] - 3s 11ms/step - loss: 2.0572 - accuracy: 0.2794
Epoch 2/6
12/12 [==============================] - 0s 21ms/step - loss: 1.3452 - accuracy: 0.7023
Epoch 3/6
12/12 [==============================] - 0s 16ms/step - loss: 0.9085 - accuracy: 0.7755
Epoch 4/6
12/12 [==============================] - 0s 15ms/step - loss: 0.6278 - accuracy: 0.8355
Epoch 5/6
12/12 [==============================] - 0s 12ms/step - loss: 0.4784 - accuracy: 0.8877
Epoch 6/6
12/12 [==============================] - 0s 12ms/step - loss: 0.4193 - accuracy: 0.9138


In [7]:
# Una vez que haya terminado el entrenamiento, guardar el modelo nuevamente si es necesario
retrained_model.save("model_retrained.h5")

# Reentrenamiento de mix_model.h5. Utiliza el model.h5 preentrenado con MNIST, y reentrena las últimas capas con los datos extraídos de la app

In [8]:
mnist_model = keras.models.load_model("model_mnist.h5")
mix_model = keras.models.clone_model(mnist_model)
mix_model.set_weights(mnist_model.get_weights())
for layer in mix_model.layers[:-12]: #las últimas tres capas
    layer.trainable = False

mix_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

mix_model.fit(np.array(train_vectors), labels, epochs=6)

Epoch 1/6
12/12 [==============================] - 3s 9ms/step - loss: 2.0237 - accuracy: 0.7023
Epoch 2/6
12/12 [==============================] - 0s 9ms/step - loss: 1.0233 - accuracy: 0.8016
Epoch 3/6
12/12 [==============================] - 0s 9ms/step - loss: 0.7519 - accuracy: 0.8094
Epoch 4/6
12/12 [==============================] - 0s 9ms/step - loss: 0.5426 - accuracy: 0.8773
Epoch 5/6
12/12 [==============================] - 0s 14ms/step - loss: 0.3725 - accuracy: 0.8903
Epoch 6/6
12/12 [==============================] - 0s 14ms/step - loss: 0.2803 - accuracy: 0.9138


In [9]:
# Una vez que haya terminado el entrenamiento, guardar el modelo
mix_model.save("model_mix.h5")

# Entrenamiento del modelo con conjunto de datos mnist

In [86]:
"""
# Cargar el conjunto de datos MNIST
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Normalizar los valores de píxeles al rango [0, 1]
train_images, test_images = train_images / 255.0, test_images / 255.0

# Crear un modelo secuencial
mnist_model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

# Compilar el modelo
mnist_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
mnist_model.fit(np.array(train_images), train_labels, epochs=6)

# Una vez que haya terminado el entrenamiento, guardar el modelo
mnist_model.save("model_mnist.h5")
"""

Epoch 1/6
1875/1875 [==============================] - 24s 11ms/step - loss: 0.2962 - accuracy: 0.9146
Epoch 2/6
1875/1875 [==============================] - 26s 14ms/step - loss: 0.1412 - accuracy: 0.9583
Epoch 3/6
1875/1875 [==============================] - 15s 8ms/step - loss: 0.1079 - accuracy: 0.9672
Epoch 4/6
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0880 - accuracy: 0.9725
Epoch 5/6
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0756 - accuracy: 0.9763
Epoch 6/6
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0657 - accuracy: 0.9790


***
# Comparativa de modelos
***

In [111]:
mnist_model = keras.models.load_model("model_mnist.h5")
mix_model = keras.models.load_model("model_mix.h5")
retrainedt_model = keras.models.load_model("model_retrained.h5")